*UE Learning from User-generated Data, CP MMS, JKU Linz 2025*
# Exercise 6: Content based Filtering

In this exercise, we delve into content-based filtering, a type of recommender system that makes recommendations by utilizing the features of items and a profile of the user's preferences. Unlike collaborative filtering, which relies on the user-item interactions, content-based filtering focuses on the properties of the items themselves to make recommendations. This approach is particularly useful when we have detailed metadata about items or when dealing with cold start problems related to new users or items.

An example of item content properties could be (for a music track): artist name, track title, year of release, genre, as well as the audio track itself (remember, in the collaborative filtering scenario we only worked with item ids without caring about what they actually were). Such complex information as the audio track is usually handled in a form of (item) embeddings, high-dimensional vector representations that capture the characteristics of each item. By analyzing these embeddings, we can identify items that are similar to those a user has liked in the past and recommend them accordingly.

Please consult the lecture slides on content-based filtering for a recap.

Make sure to rename the notebook according to the convention:

LUD25_ex06_k<font color='red'><Matr. Number\></font>_<font color='red'><Surname-Name\></font>.ipynb

for example:

LUD25_ex06_k000007_Bond_James.ipynb

## Implementation
In this exercise, you will implement two content-based filtering algorithms using item embeddings. We provide the embeddings for each item, and your task is to find items most similar to a user's consumption history. You will then evaluate the performance of your algorithms using the normalized Discounted Cumulative Gain (nDCG) metric across different user groups.

Please **only use libraries already imported in the notebook**.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity as cosine_similarity
from rec import inter_matr_implicit
from tqdm import tqdm
from typing import Callable, List

## Data Overview and Loading

This exercise utilizes a dataset that consists of user-item interactions and item embeddings. The dataset is split into several files:

`*.user`: Contains information about users.
`*.item`: Contains information about items.
`*.inter_train`: Contains user-item interactions used for training the recommender system.
`*.inter_test`: Contains user-item interactions held out for testing the recommender system.
`*.id_musicnn`: Contains embeddings for each item.

Let's start by loading these files and taking a closer look at their content.

In [2]:
def read(dataset, file):
    return pd.read_csv(dataset + '/' + dataset + '.' + file, sep='\t')

# Load User Data
users = read('lfm-tiny-tunes', 'user')
print("Users Data Head:")
print(users.head())

# Load Item Data
items = read('lfm-tiny-tunes', 'item')
print("\nItem Data Head:")
print(items.head())

# Load Training Interactions
train_inters = read('lfm-tiny-tunes', 'inter_train')
print("\nTraining Interactions Head:")
print(train_inters.head())

# Load Testing Interactions
test_inters = read('lfm-tiny-tunes', 'inter_test')
print("\nTesting Interactions Head:")
print(test_inters.head())

# Load Embeddings
embedding = read('lfm-tiny-tunes', 'id_musicnn')
print("\nEmbeddings Head:")
print(embedding.head())

train_interaction_matrix = inter_matr_implicit(users, items, train_inters, 'lfm-tiny-tunes')
test_interaction_matrix = inter_matr_implicit(users, items, test_inters, 'lfm-tiny-tunes')

Users Data Head:
   user_id country  age_at_registration gender    registration_date
0        0      RU                   25      m  2006-06-12 13:25:12
1        1      US                   23      m  2005-08-18 15:25:41
2        2      FR                   25      m  2006-02-26 22:39:03
3        3      DE                    2      m  2007-02-28 10:12:13
4        4      UA                   23      n  2007-10-09 15:21:20

Item Data Head:
                artist                                   track  item_id
0           Black Flag                              Rise Above        0
1                 Blur  For Tomorrow - 2012 Remastered Version        1
2          Damien Rice                            Moody Mooday        2
3                 Muse                            Feeling Good        3
4  My Bloody Valentine                                    Soon        4

Training Interactions Head:
   user_id  item_id  listening_events
0      510       50                 3
1      510      324  

Item Similarity Calculation

Here you can experiment with the cosine similarity between two item embeddings, to get a feeling what sensible inputs and outputs are. Cosine similarity is a measure that calculates the cosine of the angle between two vectors, often used to measure item similarity in recommendation systems. We will use this later.

In [3]:
embedding_x = np.array([0.1, 0.2, 0.3, 0.4]).reshape(1, -1)
embedding_y = np.array([0.0, 0.1, 0.1, -0.1]).reshape(1, -1)
similarity = cosine_similarity(embedding_x, embedding_y)
print(f"Cosine Similarity: {similarity}")
assert -1 <= similarity <= 1, "Cosine similarity is out of bounds."

Cosine Similarity: [[0.10540926]]


## <font color='red'>TASK 1/3</font>: Implementing the Average Embedding Similarity Recommender

The idea of the first content-based recommender is to use the embeddings of the items the user already consumed to create one representation of the user's taste (by just averaging them). Then the recommedation score is assigned to each of the items in the collection as cosine_similarity between the user's profile and each item's embedding. Highest scoring items not present in the user's history are then selected for recommendation.

First, familiarize yourself with the concept of item embeddings, which are vector representations capturing the essential features or qualities of each item.

After that, develop a function that takes a user's interaction history (items they have already interacted with/seen) and the item embeddings as input. This function should:

* Create the user's profile embedding as discussed, calculate the similarity between the profile and all items in the collection.
* Rank the items based on their similarity scores.
* To ensure that only unseen items are recommended, set the aggregated similarity scores of the items the user has already interacted with to **0** as a floating point number. This will effectively remove them from consideration during the ranking process.
* Recommend the top-K most similar items that the user has not interacted with yet.

In [4]:
def calculate_user_profile_embedding(seen_item_ids: list, item_embeddings: pd.DataFrame) -> np.ndarray:
    """
    Calculates the average embedding of items a user has interacted with to create a user profile embedding.

    Parameters:
    - seen_item_ids - list[int], IDs of items already seen by the user, used to filter the embeddings;
    - item_embeddings - pd.DataFrame, Unsorted DataFrame containing item_id and item embeddings as separate columns;

    Returns:
    - np.ndarray: numpy array of shape (1,embedding_dim), representing the user's average embedding profile. 
    """
    
    user_profile_embedding = None
    
    # TODO: YOUR IMPLEMENTATION

    # Identifying id and the embedding columns
    id_col = 'item_id' if 'item_id' in item_embeddings.columns else item_embeddings.columns[0]
    embed_cols = item_embeddings.columns.drop(id_col)

    # Taking user already seen item
    seen_rows = item_embeddings[item_embeddings[id_col].isin(seen_item_ids)][embed_cols]

    if seen_rows.empty:
        return np.zeros((1, len(embed_cols)))
    
    user_profile_embedding = seen_rows.to_numpy(dtype=float).mean(axis=0, keepdims=True)
    
    return user_profile_embedding


def average_embedding_similarity_rec(seen_item_ids: list, item_embeddings: pd.DataFrame, _calculate_user_profile_embedding: Callable[[List[int], pd.DataFrame], np.ndarray], top_k: int=10) -> np.ndarray:
    """
    Recommends items to a user based on the average embedding similarity of items they have interacted with.
    It computes the cosine similarity between the user profile and all other items, recommending
    the top-K most similar items that the user has not yet interacted with.

    seen_item_ids - list[int], ids of items already seen by the user (to exclude from recommendation);
    embedding - pd.DataFrame, Unsorted DataFrame containing item_id and item embeddings as separate columns;
    _calculate_user_profile_embedding - function, function to calculate the user's average embedding profile;
    topK - int, number of recommendations per user to be returned;

    returns - 1D np.ndarray, array of IDs of the top-K recommended items, sorted by decreasing similarity
            to the user's average embedding profile;
    """

    recommended_item_ids = None
    user_profile_embedding = None

    # TODO: YOUR IMPLEMENTATION

    id_col = 'item_id' if 'item_id' in item_embeddings.columns else item_embeddings.columns[0]
    embed_cols = item_embeddings.columns.drop(id_col)

    user_profile_embedding = calculate_user_profile_embedding(seen_item_ids, item_embeddings)

    # Similar profile every item 
    item_matrix = item_embeddings[embed_cols].to_numpy(dtype=float)
    sims = cosine_similarity(user_profile_embedding, item_matrix).flatten()

    # Mask out the things already consumed
    seen_mask = item_embeddings[id_col].isin(seen_item_ids).to_numpy()
    sims[seen_mask] = 0.0

    # Pick the top-k
    k = min(top_k, len(sims))
    top_idx = np.argpartition(-sims, k - 1)[:k]   
    top_idx = top_idx[np.argsort(-sims[top_idx])]     
    recommended_item_ids = item_embeddings.iloc[top_idx][id_col].to_numpy()

    return recommended_item_ids

In [5]:
user_id_example = users['user_id'].iloc[1]
seen_item_ids = train_inters[train_inters['user_id'] == user_id_example]['item_id'].values.tolist()
recommended_items = average_embedding_similarity_rec(seen_item_ids, embedding, calculate_user_profile_embedding, top_k=5)
print(f"Recommended Items for User {user_id_example}: {recommended_items}")

Recommended Items for User 1: [244  53 259 201 293]


## <font color='red'>TASK 2/3</font>: Implementing a Сontent-based ItemKNN

In this task, you have to implement an ItemKNN (similar to exercise 2) but this time using content-based features. This technique builds upon the concept of item embeddings and similarity calculations, but instead of creating a single user profile, it considers the individual similarities between each item the user has interacted with and the top-(KNN)-k items. This allows for a more diverse set of recommendations that capture different aspects of the user's preferences.

**Tips on implementation:**
* For each item the user has interacted with, calculate its similarity to all other items in the dataset using their embeddings (using cosine similarity), then keep only the top-k items. Make sure to sort the item embeddings by item_id before calling ```compute_itemknn_scores```.
* For each item that could potentially be recommended, combine the similarity scores it received from each of the items the user has already interacted with. This combined score will reflect the overall similarity of the potential recommendation to the user's preferences as expressed through their past interactions. Keep track of the highest similarity score encountered for each potential recommendation.
* To ensure that only unseen items are recommended, set the aggregated similarity scores of the items the user has already interacted with to negative infinity. This will effectively remove them from consideration during the ranking process.
* The function ```compute_itemknn_scores``` returns an array (sorted by item_id, not by value) of aggregated similarity scores for all items, with higher scores indicating higher similarity.
* Rank the items based on their aggregated similarity scores. Recommend the top-K items that the user has not interacted with yet.

In [6]:
def compute_itemknn_scores(seen_item_ids: list, item_embeddings: pd.DataFrame, k: int = 10) -> np.ndarray:
    """
    ItemKNN-like scoring using item embeddings
    
    For each item, find its k most similar items (based on embeddings),
    and if any of those have been seen by the user, use their similarities
    to compute an aggregated score.
    
    seen_item_ids - list[int], items the user has seen
    item_embeddings - pd.DataFrame, must include 'item_id' and embedding columns
    k - int, number of nearest neighbors to consider
    
    returns - np.ndarray of scores for each item in item_embeddings
    """
    
    recommendation_scores = None
    
    # TODO: YOUR IMPLEMENTATION

    id_col = 'item_id' if 'item_id' in item_embeddings.columns else item_embeddings.columns[0]
    embed_cols = item_embeddings.columns.drop(id_col)

    item_ids = item_embeddings[id_col].to_numpy()
    emb_matrix = item_embeddings[embed_cols].to_numpy(dtype=float)

    # Mapping the id to row index (safety)
    id2idx = {iid: idx for idx, iid in enumerate(item_ids)}
    seen_indices = [id2idx[i] for i in seen_item_ids if i in id2idx]

    # Inizialization to -inf, so everything missing stay unrankend
    recommendation_scores = np.full(len(item_ids), -np.inf)

    if not seen_indices: 
        return recommendation_scores

    # Similarity: all seen items x full catalogue
    sim_matrix = cosine_similarity(emb_matrix[seen_indices], emb_matrix)

    # Keeping the k closest neighbours per seen item
    for row, sidx in zip(sim_matrix, seen_indices):
        row[sidx] = -np.inf # --> ignore self-similarity
        neighbour_idx = np.argpartition(-row, k)[:k] if k < len(row) else np.arange(len(row))
        
        # update global score array with the MAX similarity encountered
        np.maximum.at(recommendation_scores, neighbour_idx, row[neighbour_idx])

    # Never recommend already-seen item
    recommendation_scores[seen_indices] = -np.inf

    return recommendation_scores


def cb_itemknn_recommendation(seen_item_ids: list, item_embeddings: pd.DataFrame, 
                              _compute_itemknn_scores: Callable[[List[int], pd.DataFrame], np.ndarray], 
                              top_k: int=10, knn_k: int=10) -> np.ndarray:
    """
    Recommends items to a user based on the items they have already seen, by sorting the calculated similarity scores
    and selecting the top-k items.

    seen_item_ids - list[int], ids of items already seen by the user (to exclude from recommendation);
    embedding - pd.DataFrame, Unsorted DataFrame containing item_id and item embeddings as separate columns;
    _compute_itemknn_scores - function, function to compute aggregated similarity scores for all items;
    topK - int, number of recommendations per user to be returned;

    returns - 1D np.ndarray, array of IDs of the top-K recommended items, sorted by decreasing similarity
            to the user's average embedding profile;
    """

    recommended_item_ids = None
    # TODO: YOUR IMPLEMENTATION

    item_embeddings = item_embeddings.sort_values(
        'item_id' if 'item_id' in item_embeddings.columns else item_embeddings.columns[0]
    ).reset_index(drop=True)

    scores = _compute_itemknn_scores(seen_item_ids, item_embeddings, k=knn_k)

    # filter out –inf
    valid_mask = np.isfinite(scores)
    if not np.any(valid_mask):
        return np.array([], dtype=int)

    k = min(top_k, valid_mask.sum())
    top_idx = np.argpartition(-scores, k - 1)[:k]
    top_idx = top_idx[np.argsort(-scores[top_idx])]   

    id_col = 'item_id' if 'item_id' in item_embeddings.columns else item_embeddings.columns[0]

    recommended_item_ids = item_embeddings.iloc[top_idx][id_col].to_numpy()
    
    return recommended_item_ids


In [7]:
user_id_example = users['user_id'].iloc[1]
seen_items_user_example = np.where(train_interaction_matrix[user_id_example, :] > 0)[0]
recommended_items = cb_itemknn_recommendation(seen_items_user_example.tolist(), embedding, compute_itemknn_scores, top_k=5)
print(f"Recommended Items for User {user_id_example}: {recommended_items}")

Recommended Items for User 1: [183  43 167 246 202]


## <font color='red'>TASK 3/3</font>: Evaluating Recommendations with nDCG

In this task, you will evaluate the performance of the content-based filtering algorithm you've implemented, alongside other recommender systems that utilize collaborative filtering. Specifically, you will use Precision@K, Recall@K and the normalized Discounted Cumulative Gain (nDCG) metric to assess how effective each recommender system is across different user groups based on their interaction levels.

You will compare the following recommender systems:

| Recommender Name                      | Abbreviation     | Relative Time Complexity     |
|---------------------------------------|------------------|---------------------|
| Average Embedding Similarity           | Avg_Item_Embd    |$ O(n^2)   $         |
| Content-based Item K-Nearest Neighbors                            | CB_ItemKNN       | $O(n^2) $             |
| Singular Value Decomposition         | SVD              | $O(n * m * k)$        |
| Collaborative Filtering Item K-Nearest Neighbors             | CF_ItemKNN          | $O(n^2)$              |
| Top Popular                          | TopPop           | $O(n \log{n}) $         |

In [8]:
from rec import svd_decompose, svd_recommend_to_list
from rec import recTopK
from rec import recTopKPop
from sklearn.metrics import ndcg_score, precision_score, recall_score

In [9]:
def evaluation_metrics(recs, g_truth):
    predicted_scores = np.zeros(g_truth.shape[1])
    predicted_scores[recs] = np.arange(len(recs), 0, -1)
    
    predicted_scores_binary = np.zeros(g_truth.shape[1])
    predicted_scores_binary[recs] = 1
    
    score_ndcg = ndcg_score(g_truth, predicted_scores.reshape(1, -1), k=len(recs))
    p_a_score = precision_score(g_truth[0],predicted_scores_binary)
    r_a_score = recall_score(g_truth[0],predicted_scores_binary)     
    
    return p_a_score, r_a_score, score_ndcg

def evaluate_recommender_by_user_groups(user_groups: dict, recommenders: dict, train_interaction_matrix: np.ndarray, test_interaction_matrix: np.ndarray,
                                 U: np.ndarray, V: np.ndarray, item_embeddings: pd.DataFrame, _calculate_user_profile_embedding, 
                                 _compute_aggregated_scores, topK: int=10, n_neighbors: int=5) -> pd.DataFrame:
    """
    Evaluates recommender systems across user groups, calculating average evaluation metrics (Precision@K, Recall@K, nDCG).

    user_groups - dict, keys - names of the user groups (str), values - lists of user IDs belonging to each group;
    recommenders - dict, keys - names of recommenders (str), values - recommender functions;
    train_interaction_matrix - 2D np.ndarray (users x items), interaction matrix from the training set;
    test_interaction_matrix - 2D np.ndarray (users x items), interaction matrix from the test set;
    U, V - 2D np.ndarray, matrices resulting from SVD decomposition of the interaction matrix;
    item_embeddings - pd.DataFrame, DataFrame containing item IDs and their embeddings;
    topK - int, number of top recommendations to consider for evaluation;
    n_neighbors - int, number of neighbors for ItemKNN recommender;

    returns - pd.DataFrame, with columns: 'User Group', 'Recommender', 'Average p@k', 
              'Average r@k','Average nDCG', containing evaluation results;
    """
    results = []

    for group_name, users in user_groups.items():
        for recommender_name, recommender_func in tqdm(recommenders.items(), desc=f'Evaluating {group_name} Users'):
            metric_scores = np.zeros((len(users),3))
            for i, user_id in enumerate(users):
                seen_items = np.where(train_interaction_matrix[user_id, :] > 0)[0]  # Items already interacted with by the user
                if recommender_name == 'SVD':
                    recommendations = recommender_func(user_id, seen_items.tolist(), U, V, topK)
                elif recommender_name == 'CF_ItemKNN':
                    recommendations = recommender_func(train_interaction_matrix, user_id, topK, n_neighbors)
                elif recommender_name == 'TopPop':
                    recommendations = recommender_func(train_interaction_matrix, user_id, topK)
                elif recommender_name == 'Avg_Item_Embd':
                    recommendations = recommender_func(seen_items.tolist(), item_embeddings, _calculate_user_profile_embedding, topK)
                elif recommender_name == 'CB_ItemKnn':
                    recommendations = recommender_func(seen_items.tolist(), item_embeddings, _compute_aggregated_scores, topK)
                else:
                    raise NotImplementedError(f'Recommender {recommender_name} not implemented.')

                if not isinstance(recommendations, np.ndarray):
                    recommendations = np.array(recommendations)

                true_relevance = test_interaction_matrix[user_id, :].reshape(1, -1)
                metric_scores[i,:] = evaluation_metrics(recommendations, true_relevance)

            metric_scores_avg = np.mean(metric_scores,axis = 0)
            pk_score, rk_score, score_ndcg = metric_scores_avg
            
            results.append({'User Group': group_name, 'Recommender': recommender_name, 
                            'Average p@k': pk_score, 'Average r@k': rk_score, 
                            'Average nDCG': score_ndcg,})

    return pd.DataFrame(results)

Here, you will implement a function that evaluates the performance of the recommenders across different user groups based on their interaction levels. You will need to split the users into two groups: one with low interaction levels (below or equal a certain threshold) and one with high interaction levels (above the threshold). The function should then call the `evaluate_ndcg_by_user_groups` function to calculate the average nDCG scores for each recommender across the user groups. Make sure to only use the passed variables and parameters in your implementation.

In [10]:
def evaluate_recommenders(_evaluate_recommender_by_user_groups, user_info: pd.DataFrame, parameters: dict, recommender: dict, user_threshold: int) -> (pd.DataFrame, dict):
    """
    Evaluates recommenders across user groups based on interaction levels.

    Splits users into low and high interaction groups based on a threshold and calculates
    average Precision@K, Recall@K, and nDCG scores for each recommender within each group.

    _evaluate_recommender_by_user_groups - function, function to evaluate recommenders across user groups;
    user_info - pd.DataFrame, DataFrame containing user information.
    parameters - dict, Dictionary containing data and parameters for evaluation, including:
        train_interaction_matrix - 2D np.ndarray, test_interaction_matrix - 2D np.ndarray,
        U - 2D np.ndarray, V - 2D np.ndarray, item_embeddings - pd.DataFrame, topK - int,
        n_neighbors - int.
    recommender - dict, Dictionary of recommender functions, with keys as recommender names
                        and values as the corresponding functions.
    user_threshold - int, Threshold for dividing users into low and high interaction groups.

    returns - tuple:
        pd.DataFrame, DataFrame containing evaluation results with columns: 'User Group',
            'Recommender', 'Average nDCG'.
        dict, Dictionary containing the user groups with keys 'Low Interaction' and
            'High Interaction', and values as lists of user IDs.

    """

    evaluation_results_df = None

    user_groups = {
        'Low Interaction': [],
        'High Interaction': []
    }

    # TODO: YOUR IMPLEMENTATION

    # Unpacking dictionary
    train_inter = parameters['train_interaction_matrix']
    n_users     = train_inter.shape[0]

    # Mapping row index - user_id mapping
    if 'user_id' in user_info.columns:
        user_ids = user_info['user_id'].to_numpy()
    else:
        user_ids = np.arange(n_users)

    for row_idx, uid in enumerate(user_ids):
        n_inter = np.count_nonzero(train_inter[row_idx, :])
        if n_inter <= user_threshold:
            user_groups['Low Interaction'].append(row_idx)
        else:
            user_groups['High Interaction'].append(row_idx)

    # Evaluation routine
    evaluation_results_df = _evaluate_recommender_by_user_groups(
        user_groups          = user_groups,
        recommenders         = recommender,
        train_interaction_matrix = parameters['train_interaction_matrix'],
        test_interaction_matrix  = parameters['test_interaction_matrix'],
        U = parameters['U'],
        V = parameters['V'],
        item_embeddings = parameters['item_embeddings'],
        _calculate_user_profile_embedding = parameters['_calculate_user_profile_embedding'],
        _compute_aggregated_scores        = parameters['_compute_itemknn_scores'],
        topK       = parameters['topK'],
        n_neighbors= parameters['n_neighbors']
    )

    return evaluation_results_df, user_groups

The following Cell will evaluate the implemented recommenders on the given dataset. The evaluation results will be displayed in a DataFrame, showing the average nDCG scores for each recommender across different user groups. This Cell is for you to see how the input looks like. For a correct evaluation, the code below needs to run without errors and the nDCG scores need to be output as described.

In [11]:
# Define recommenders with correct parameters
recommenders = {
    'Avg_Item_Embd': average_embedding_similarity_rec,
    'CB_ItemKnn': cb_itemknn_recommendation,
    'SVD': svd_recommend_to_list,
    'CF_ItemKNN': recTopK,
    'TopPop': recTopKPop
}

U, V = svd_decompose(train_interaction_matrix)

data = {
    'train_interaction_matrix': train_interaction_matrix,
    'test_interaction_matrix': test_interaction_matrix,
    'U': U,
    'V': V,
    'item_embeddings': embedding,
    '_calculate_user_profile_embedding': calculate_user_profile_embedding,
    '_compute_itemknn_scores': compute_itemknn_scores,
    'topK': 10,
    'n_neighbors': 5}

evaluation_results_df, user_groups = evaluate_recommenders(evaluate_recommender_by_user_groups, users, data, recommenders, user_threshold=5)
print(f"Number of Users with low interaction levels: {len(user_groups['Low Interaction'])}")
print(f"Number of Users with high interaction levels: {len(user_groups['High Interaction'])}")
print(evaluation_results_df)

Evaluating High Interaction Users: 100%|██████████| 5/5 [00:34<00:00,  6.97s/it]

Number of Users with low interaction levels: 560
Number of Users with high interaction levels: 655
         User Group    Recommender  Average p@k  Average r@k  Average nDCG
0   Low Interaction  Avg_Item_Embd     0.006964     0.042857      0.022220
1   Low Interaction     CB_ItemKnn     0.010357     0.058036      0.033966
2   Low Interaction            SVD     0.027500     0.187500      0.129047
3   Low Interaction     CF_ItemKNN     0.039643     0.269643      0.169657
4   Low Interaction         TopPop     0.015893     0.121429      0.069482
5  High Interaction  Avg_Item_Embd     0.016794     0.051901      0.036076
6  High Interaction     CB_ItemKnn     0.019389     0.063305      0.040686
7  High Interaction            SVD     0.056031     0.190630      0.154937
8  High Interaction     CF_ItemKNN     0.087328     0.303126      0.236496
9  High Interaction         TopPop     0.046870     0.162379      0.115514


In [12]:
# The end.